In [84]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/bajlecgrausayegh/keras/default/1/config.json
/kaggle/input/bajlecgrausayegh/keras/default/1/metadata.json
/kaggle/input/bajlecgrausayegh/keras/default/1/model.weights.h5
/kaggle/input/dl-itba-cifar-100-2024-q-1/y_train_fine.npy
/kaggle/input/dl-itba-cifar-100-2024-q-1/y_train_coarse.npy
/kaggle/input/dl-itba-cifar-100-2024-q-1/fine_label_names.pck
/kaggle/input/dl-itba-cifar-100-2024-q-1/coarse_label_names.pck
/kaggle/input/dl-itba-cifar-100-2024-q-1/x_test.npy
/kaggle/input/dl-itba-cifar-100-2024-q-1/x_train.npy


In [85]:
from matplotlib import pyplot as plt
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [86]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Flatten, Conv2D, MaxPool2D, GlobalAveragePooling2D, Activation
from tensorflow.keras.optimizers import SGD, Adam, RMSprop
from tensorflow.keras.regularizers import l1, l2
from tensorflow.keras.constraints import MaxNorm
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint, TensorBoard
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from tensorflow.keras import models
from tensorflow.keras import layers, models, regularizers, constraints

In [87]:
x_train = np.load("/kaggle/input/dl-itba-cifar-100-2024-q-1/x_train.npy")
x_test = np.load("/kaggle/input/dl-itba-cifar-100-2024-q-1/x_test.npy")
y_train_coarse = np.load("/kaggle/input/dl-itba-cifar-100-2024-q-1/y_train_coarse.npy")
y_train_fine = np.load("/kaggle/input/dl-itba-cifar-100-2024-q-1/y_train_fine.npy")

In [88]:
from sklearn.model_selection import train_test_split

In [89]:
import pickle
with open("/kaggle/input/dl-itba-cifar-100-2024-q-1/fine_label_names.pck", "rb") as f:
    labels_fine = pickle.load(f)
with open("/kaggle/input/dl-itba-cifar-100-2024-q-1/coarse_label_names.pck", "rb") as f:
    labels_coarse = pickle.load(f)

In [90]:
y_train_fine.shape

(50000,)

In [91]:
x_train.shape

(50000, 32, 32, 3)

In [92]:
x_train,x_val, y_train, y_val = train_test_split(x_train, y_train_fine, test_size=0.1, stratify=y_train_fine)

In [93]:
y_val.shape

(5000,)

In [94]:
x_val.shape

(5000, 32, 32, 3)

In [95]:
x_train_horizontal_flip= x_train[:,:,::-1,:]
x_train_vertical_flip = x_train[:,::-1,:,:]
x_train_vh_flip =x_train[:, ::-1, ::-1, :]

x_train_dup = np.concatenate([x_train,x_train_horizontal_flip,x_train_vertical_flip,x_train_vh_flip],axis=0)
x_train_dup = np.concatenate([x_train,x_train_horizontal_flip],axis=0)
y_train_dup = np.concatenate([y_train, y_train,y_train, y_train], axis=0)
y_train_dup = np.concatenate([y_train, y_train], axis=0)

x_train_bis = x_train_dup.copy()
x_train_bis_bis = x_train_dup.copy()

#desplazamientos right and left 
x_train_bis[:,:,1:,:] =x_train_dup[:,:,:-1,:]
x_train_bis_bis[:,:,:-1,:] =x_train_dup[:,:,1:,:]

x_train_dup = np.concatenate([x_train_dup, x_train_bis, x_train_bis_bis],axis=0)
y_train_dup = np.concatenate([y_train_dup, y_train_dup, y_train_dup], axis=0)


In [96]:
print(f'Original dataset size: {len(x_train)}')
print(f'Augmented dataset size: {len(x_train_dup)}')

Original dataset size: 45000
Augmented dataset size: 270000


In [97]:
x_train_dup.shape, y_train_dup.shape

((270000, 32, 32, 3), (270000,))

In [98]:
import keras

In [99]:
def create_model(input_shape=(32, 32, 3),l2_lambda=0.001):
    # Input layer for images
    inputs = layers.Input(shape=input_shape)
    
    # Flattening layer
    x = layers.Flatten()(inputs)
    
    # Normalization layer
    x = layers.Lambda(lambda x: x/np.float16(255.0))(x)
    
    # Dense layers with L1 regularization
    n = 9
    for i in range(3):
        x = layers.Dense(2**n,
                         kernel_regularizer=regularizers.L2(l2_lambda))(x)
        x = layers.BatchNormalization()(x)
        x = layers.ReLU()(x)
    
#     x = layers.Dense(2**(n-1), kernel_regularizer=regularizers.L2(l2_lambda))(x)
#     x = layers.BatchNormalization()(x)
#     x = layers.ReLU()(x)
    
    # Output layer with softmax for 100 classes
    fine_output = layers.Dense(100, activation='softmax', name='fine_output')(x)   
  
    # Create model
    model = models.Model(inputs=inputs, outputs=[fine_output], name=f"mlp_allnighter")
    
    return model

print('Success')

Success


In [100]:
model = create_model()
model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss={'fine_output': 'sparse_categorical_crossentropy'},
    metrics={'fine_output': 'accuracy' }
             )

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_accuracy', patience=8, restore_best_weights=True,verbose= 1)
tb = TensorBoard(log_dir="logs")

# Display the model summary
model.summary()

Model: "mlp_allnighter"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_14 (InputLayer)     │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_14 (Flatten)            │ (None, 3072)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lambda_14 (Lambda)              │ (None, 3072)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_42 (Dense)                │ (None, 512)            │     1,573,376 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_42          │ (None, 512)            │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_42 (ReLU)                 │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_43 (Dense)                │ (None, 512)            │       262,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_43          │ (None, 512)            │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_43 (ReLU)                 │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_44 (Dense)                │ (None, 512)            │       262,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_44          │ (None, 512)            │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_44 (ReLU)                 │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fine_output (Dense)             │ (None, 100)            │        51,300 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,156,132 (8.22 MB)

 Trainable params: 2,153,060 (8.21 MB)

 Non-trainable params: 3,072 (12.00 KB)

In [101]:
# Fit the model
history = model.fit(x=x_train_dup,   
                    y={'fine_output': y_train_dup},
                    validation_data=(x_val, {'fine_output': y_val}),
                    epochs=50,                        
                    batch_size=512,  # approximately 1000 batches per epoch                    
                    callbacks=[tb, early_stopping])

Epoch 1/50
528/528 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - accuracy: 0.1385 - loss: 5.7209 - val_accuracy: 0.1956 - val_loss: 5.1166
Epoch 2/50
528/528 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.2750 - loss: 4.6237 - val_accuracy: 0.2294 - val_loss: 4.7075
Epoch 3/50
528/528 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.3444 - loss: 4.0743 - val_accuracy: 0.2490 - val_loss: 4.4465
Epoch 4/50
528/528 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.4033 - loss: 3.6455 - val_accuracy: 0.2458 - val_loss: 4.3264
Epoch 5/50
528/528 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.4611 - loss: 3.2819 - val_accuracy: 0.2466 - val_loss: 4.2345
Epoch 6/50
528/528 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.5152 - loss: 2.9619 - val_accuracy: 0.2638 - val_loss: 4.1471
Epoch 7/50
528/528 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.5688 - loss: 2.6782 - val_accuracy: 0.2560 - val_loss: 4.1976
Epoch 8/50
528/528 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.6213 - loss: 2.4218 - val_accuracy: 0.

In [102]:
# Save the model
#model.save("accuracy0.2_manylayers_batch_norm2.keras")

In [103]:
predictions = model.predict(x_test).argmax(axis=1)
df = pd.DataFrame(predictions, columns=["Label"])

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


In [104]:
df

,Label
0,12
1,42
2,59
3,51
4,71
...,...
9995,77
9996,21
9997,51
9998,50


In [105]:
df.index.name = "Id"

In [106]:
df.to_csv("submission.csv")